# STATISTICAL MODELLING

### Assume that the claims count random variable N has a Poisson distribution with given years at risk v > 0 and expected frequency λ > 0. We aim at modeling the expected frequency λ > 0 such that it allows us to incorporate structural differences (heterogeneity) or systematic effects, between different insurance policies and risks.
### v measures the volume of the aggregated portfolio. Aggregation property says that the aggregated portfolio has a compound Poisson distribution (with volume weighted expected frequency)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from patsy import dmatrices
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

In [3]:
claimsdf = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/ProyectoFinal/Claims-Frequency-Predictions/Notebooks/claimsdf_1.csv')

In [4]:
claimsdf.head()

,ClaimNb,Exposure,Area,BonusMalus,VehBrand,Region,empirical_frequencies,VehGas_Regular,VehPower_,VehAge_,DrivAge_,log_density
0,1,0.10,4,50,9,1,10.000000,1,5,1,6,7.104144
1,1,0.77,4,50,9,1,1.298701,1,5,1,6,7.104144
2,1,0.75,2,50,9,5,1.333333,0,6,2,6,3.988984
3,1,0.09,2,50,9,7,11.111111,0,7,1,5,4.330733
4,1,0.84,2,50,9,7,1.190476,0,7,1,5,4.330733


In [5]:
claimsdf.drop(columns=['empirical_frequencies'], inplace=True)

## Poisson-GENERALIZED LINEAR MODEL

- The feature components interact in a multiplicative way in our Poisson GLM. One of the main tasks is to analyze whether this multiplicative interaction is appropriate. For GLM modeling approach, as the frequencies are non-linearly related to Vehicle Age and Driver Age as we've seen in the EDA, we should partition them and then treat them as categorical variables.
- We consider 3 continuous feature components (Area, BonusMalus, log-Density), 1 binary feature component (VehGas) and 5 categorical feature components (VehPower, VehAge, DrivAge, VehBrand, Region)
- We'll dummy-encode the categorical features, in order to get a unique MLE for β
- In total, we'll get a 42 variable model.

In [6]:
claimsdf = pd.get_dummies(claimsdf, columns=['VehPower_', 'VehAge_', 'DrivAge_', 'VehBrand', 'Region'], drop_first=True)

In [7]:
claimsdf.head(2)

,ClaimNb,Exposure,Area,BonusMalus,VehGas_Regular,log_density,VehPower__5,VehPower__6,VehPower__7,VehPower__8,...,Region_4,Region_5,Region_6,Region_7,Region_8,Region_9,Region_10,Region_11,Region_12,Region_13
0,1,0.10,4,50,1,7.104144,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.77,4,50,1,7.104144,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- REFERENCE LEVEL (Variables for wich the b0 parameter accounts for): `VehPower__4`, `VehAge__1`, `DrivAge__1`, `VehBrand_1`, `Region_1.0`

### MODEL TRAINING

- We randomly (uniformly) select 80% of data for training and leave 20% for testing:

In [8]:
sample = np.random.rand(len(claimsdf)) < 0.8

In [9]:
claimsdf_train = claimsdf[sample]

In [10]:
claimsdf_train.shape

(542373, 45)

In [11]:
claimsdf_test = claimsdf[~sample]

In [12]:
claimsdf_test.shape

(135640, 45)

In [13]:
expr = """ Q('ClaimNb') ~ Q('Area') + Q('BonusMalus') + Q('log_density') + Q('VehGas_Regular') + 
                          Q('VehPower__5') + Q('VehPower__6') + Q('VehPower__7') + Q('VehPower__8') +
                          Q('VehPower__9') + Q('VehPower__10') + Q('VehPower__11') + Q('VehPower__12') + 
                          Q('VehAge__2') + Q('VehAge__3') + Q('DrivAge__2') + Q('DrivAge__3') + 
                          Q('DrivAge__4') + Q('DrivAge__5') + Q('DrivAge__6') + Q('DrivAge__7') + Q('DrivAge__8') +
                          Q('VehBrand_2') + Q('VehBrand_3') + Q('VehBrand_4') + Q('VehBrand_5') + Q('VehBrand_6') +
                          Q('VehBrand_7') + Q('VehBrand_8') + Q('VehBrand_9') + Q('VehBrand_10') + Q('VehBrand_11') +
                          Q('Region_2') + Q('Region_3') + Q('Region_4') + Q('Region_5') + Q('Region_6') + 
                          Q('Region_7') + Q('Region_8') + Q('Region_9') + Q('Region_10') + Q('Region_11') +
                          Q('Region_12') """

- Now we define the matrices for the specified model as required for the statsmodel libraries. And proceed to fit a Poisson-GLM to the TRAIN SET:

In [14]:
y_train, X_train = dmatrices(expr, claimsdf_train, return_type='dataframe')

In [15]:
poisson_model1 = sm.GLM(y_train, X_train, exposure=claimsdf_train.Exposure, family=sm.families.Poisson()).fit()

In [16]:
print(poisson_model1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           Q('ClaimNb')   No. Observations:               542373
Model:                            GLM   Df Residuals:                   542330
Model Family:                 Poisson   Df Model:                           42
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1403e+05
Date:                Wed, 06 Oct 2021   Deviance:                   1.7249e+05
Time:                        11:13:36   Pearson chi2:                 1.30e+06
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -3.4538    

- From the results of the `poisson_model1` we can see that the variables: `Area`, `VehPower__12`, `VehBrand_2`, `VehBrand_3`, `VehBrand_4`, `VehBrand_6`, `VehBrand_7`, `VehBrand_8`, `VehBrand_10`, `VehBrand_11`, `Region_12`: have a p-value bigger than 0.05 (for an assumed alpha=0.05), so we should consider its inclusion in the model, because they doesn't seem to be significant for frequency modelling.
- As we saw in the correlation analysis, `Area` is highly correlated with `Density`, wich might explain why the first is not significant.
- From the 11 `VehBrand` classes, 8 resulted non-siginificant.

#### DISPERSION:

- Deviance statistics accounts for potential over- or under-dispersion. In the Poisson model, by definition, variance should equal mean (φ = 1). We can determine this parameter empirically by Pearson’s dispersion estimate and the deviance dispersion.
- From the results of the `poisson_model1` we can see that the `scale` (Pearson's dispertion) equals to 1, wich means that we can assume that the model is not  overdispersed. 

- DEVIANCE DISPERTION:

In [96]:
sum((poisson_model1.resid_deviance) ** 2) / 542395

0.31778833768117626

- The Deviance dispertion is less than one, so we can assume that there's no overdispertion.

In [258]:
(sum((poisson_model1.resid_pearson) ** 2)) / 542395

2.4680604218140716

#### TRAINING DEVIANCE-LOSS:

In [98]:
loss_train = sum((poisson_model1.resid_deviance) ** 2) / X_train.shape[0]
loss_train

0.31780532964194214

#### AKAIKE INFORMATION CRITERION
- Akaike’s information criterion (AIC), which introduces a penalty term for over-fitting (to mimic an out-of-sample loss)

In [127]:
poisson_model1.aic

228541.37540844904

#### X2-STATISTIC: NULL DEVIANCE - RESIDUAL DEVIANCE:

In [100]:
x2_statistic = (poisson_model1.null_deviance) - sum(poisson_model1.resid_deviance)
x2_statistic

274814.06868109666

#### ROOT MEAN SQUARE ERROR OF PREDICTION:

In [279]:
y_test, X_test = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [131]:
poisson_model1_pred = poisson_model1.predict(X_test)

- Let's create a dataframe with the predctions of the Poisson GLM:

In [150]:
poisson_pred_df = pd.DataFrame(y_test)
poisson_pred_df['predicted_ClaimNb'] = poisson_model1_pred

In [178]:
rmse = []

for _ in range(10):
    p = poisson_pred_df.sample(n=135733)
    rmse_ = ((((p["Q('ClaimNb')"] - p['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse.append(rmse_)

In [188]:
poisson_rmse = sum(rmse)/len(rmse)
poisson_rmse

0.7850454180767814

## NEGATIVE BINOMIAL

In [276]:
negbi_model = sm.GLM(y_train, X_train, exposure=claimsdf_train.Exposure, family=sm.families.NegativeBinomial()).fit()

In [277]:
negbi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           Q('ClaimNb')   No. Observations:               542785
Model:                            GLM   Df Residuals:                   542742
Model Family:        NegativeBinomial   Df Model:                           42
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1391e+05
Date:                Tue, 05 Oct 2021   Deviance:                   1.5055e+05
Time:                        16:52:59   Pearson chi2:                 1.28e+06
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -3.4056      0.072    -47.295      0.000      -3.547      -3.264
Q('Area')               0.0040      0.019      0.214      0.831      -0.032       0.040
Q('BonusMalus')         0.0232      0.000     58.000      0.000       0.022       0.024
Q('log_density')        0.0378      0.014      2.721      0.007       0.011       0.065
Q('VehGas_Regular')     0.0734      0.013      5.526      0.000       0.047       0.099
Q('VehPower__5')        0.1935      0.021      9.170      0.000       0.152       0.235
Q('VehPower__6')        0.2169      0.021     10.315      0.000       0.176       0.258
Q('VehPower__7')        0.1345      0.021      6.484      0.000       0.094       0.175
Q('VehPower__8')       -0.0546      0.031     -1.755      0.079      -0.116       0.006
Q('VehPower__9')        0.2849      0.032      8.816      0.000       0.222       0.348
Q('VehPower__10')       0.2409      0.032      7.452      0.000       0.178       0.304
Q('VehPower__11')       0.1323      0.043      3.108      0.002       0.049       0.216
Q('VehPower__12')       0.0409      0.046      0.894      0.372      -0.049       0.131
Q('VehAge__2')         -0.5372      0.017    -32.209      0.000      -0.570      -0.504
Q('VehAge__3')         -0.7488      0.020    -36.535      0.000      -0.789      -0.709
Q('DrivAge__2')        -0.3797      0.053     -7.201      0.000      -0.483      -0.276
Q('DrivAge__3')        -0.5066      0.051     -9.841      0.000      -0.608      -0.406
Q('DrivAge__4')        -0.3748      0.050     -7.475      0.000      -0.473      -0.277
Q('DrivAge__5')        -0.0942      0.051     -1.857      0.063      -0.194       0.005
Q('DrivAge__6')        -0.1555      0.051     -3.029      0.002      -0.256      -0.055
Q('DrivAge__7')        -0.1903      0.053     -3.568      0.000      -0.295      -0.086
Q('DrivAge__8')        -0.0959      0.055     -1.756      0.079      -0.203       0.011
Q('VehBrand_2')         0.0073      0.018      0.412      0.680      -0.028       0.042
Q('VehBrand_3')         0.0040      0.025      0.156      0.876      -0.046       0.054
Q('VehBrand_4')        -0.0169      0.035     -0.486      0.627      -0.085       0.051
Q('VehBrand_5')         0.0632      0.029      2.170      0.030       0.006       0.120
Q('VehBrand_6')         0.0173      0.033      0.525      0.599      -0.047       0.082
Q('VehBrand_7')         0.0756      0.042      1.788      0.074      -0.007       0.159
Q('VehBrand_8')         0.1388      0.047      2.984      0.003       0.048       0.230
Q('VehBrand_9')         0.1687      0.021      8.204      0.000       0.128       0.209
Q('VehBrand_10')        0.0233      0.048      0.482      0.630      -0.071       0.118
Q('VehBrand_11')       -0.1126      0.090     -1.258      0.208      -0.288       0.063
Q('

In [282]:
negbi_model_pred = negbi_model.predict(X_test)

In [283]:
negbi_model_pred_df = pd.DataFrame(y_test)
negbi_model_pred_df['predicted_ClaimNb'] = negbi_model_pred

In [291]:
rmse_ng = []

for _ in range(10):
    ng = negbi_model_pred_df.sample(n=135200)
    rmseng = ((((ng["Q('ClaimNb')"] - ng['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse_ng.append(rmseng)

In [293]:
negbi_rmse = sum(rmse_ng)/len(rmse_ng)
negbi_rmse

0.7905599262737816

## Zero Inflated Poisson-GENERALIZED LINEAR MODEL

In [192]:
zip_model = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train, exposure=claimsdf_train.Exposure, inflation='logit').fit()

/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.210410
         Iterations: 35
         Function evaluations: 41
         Gradient evaluations: 41


/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [193]:
print(zip_model.summary())

                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:            Q('ClaimNb')   No. Observations:               542280
Model:             ZeroInflatedPoisson   Df Residuals:                   542237
Method:                            MLE   Df Model:                           42
Date:                 Tue, 05 Oct 2021   Pseudo R-squ.:                 0.02625
Time:                         12:17:18   Log-Likelihood:            -1.1410e+05
converged:                       False   LL-Null:                   -1.1718e+05
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
inflate_Intercept               0.4016      0.231      1.735      0.083      -0.052       0.855
inflate_Q('Area')              -0.0091      0.041     -0.222      0.824 

- From the results of the `zip_poisson_model` we can see that the next variables have a p-value bigger than 0.05 (for an assumed alpha=0.05), so we should consider its inclusion in the model:

- ZERO-INFLATION VARIABLES: The ones that the Logistic regression part of the ZIP model did't find useful for estimating the ϕ parameter:

  `Intercept`, `Area`, `log_density` 
  
  `VehPower_5`, `VehPower_6`, `VehPower__7`, `VehPower_9`, `VehPower_10`, `VehPower_11`,  `VehPower_12`
  
  `DrivAge_2`, `DrivAge_3`, `DrivAge_4`, `DrivAge_5`, `DrivAge_6`, `DrivAge_7`, `DrivAge_8`
  
  `VehBrand_3`, `VehBrand_4`, `VehBrand_5`, `VehBrand_6`, `VehBrand_7`, `VehBrand_8`, `VehBrand_9`, `VehBrand_10`, 
  `VehBrand_11` 
  
  `Region_2`, `Region_3`, `Region_4`, `Region_5`, `Region_6`, `Region_9`, `Region_10`, `Region_11`
  

- VARIABLES did't find useful by the Poisson part of the ZIP model to estimate ClaimNb:

   `VehBrand_3`, `VehBrand_4`, `VehBrand_5`, `VehBrand_7`, `VehBrand_8`, `VehBrand_9`, `VehBrand_10`,       
   `VehBrand_11`
    
   `Region_2`, `Region_3`, `Region_6`, `Region_9`, `Region_10`, `Region_11`
 
   `DrivAge_5`, `DrivAge_6`, `DrivAge_7`, `DrivAge_8`
 
   `VehPower__7`, `VehPower_11`,  `VehPower_12`
 

- For the logistic part, we'll cut: 
  `Intercept`, `Area`, `log_density`, `DriveAge` (rejected 7 out 8 classes), `Vehicle Brand` (rejected 9 of 11 classes), `Region` (rejected 8 out of 13), and `Veh_Power` (7 out of 9 classes)

- For the Poisson part, we'll cut only `Vehicle Brand` (rejected 8 of 11 classes), wich is coincident with the results of the full Poisson model. We'll keep `Drivers Age`, although the rejected classes are the ones that account for the less observed frequencies. With respect to `Region`, we'll keep it because some of the rejected classes account for higer observe frequencies values, maybe the rejection come from the fact that there's a slightly correlation bewtween `Region` and `log_density` and `Area`.

#### AKAIKE INFORMATION CRITERION

In [194]:
zip_model.aic

228288.53644409162

#### ROOT MEAN SQUARE ERROR OF PREDICTION:

In [196]:
y_test, X_test = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [204]:
zip_model_pred = zip_model.predict(X_test, exog_infl=X_test, exposure=claimsdf_test.Exposure)

- Let's create a dataframe with the predctions of the Zip GLM:

In [207]:
zip_pred_df = pd.DataFrame(y_test)
zip_pred_df['predicted_ClaimNb'] = zip_model_pred

In [214]:
rmse_zip = []
for _ in range(10):
    zip_ = zip_pred_df.sample(n=135733)
    rmse_ = ((((zip_["Q('ClaimNb')"] - zip_['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse_zip.append(rmse_)    

In [216]:
zip_rmse = sum(rmse_zip)/len(rmse_zip)
zip_rmse

0.7450743707755763

In [1]:
zip_pred_df

NameError: name 'zip_pred_df' is not defined

### ZIP 2

In [ ]:
"""Q('Region_2') + Q('Region_3') + Q('Region_4') + Q('Region_5') + Q('Region_6') + 
 Q('Region_7') + Q('Region_8') + Q('Region_9') + Q('Region_10') + Q('Region_11') +
Q('Region_12') + Q('DrivAge__2') + Q('DrivAge__3') + 
Q('DrivAge__4') + Q('DrivAge__5') + Q('DrivAge__6') + Q('DrivAge__7') + Q('DrivAge__8')"""

In [221]:
expr = """ Q('ClaimNb') ~ Q('Area') + Q('BonusMalus') + Q('log_density') + Q('VehGas_Regular') + 
                          Q('VehPower__5') + Q('VehPower__6') + Q('VehPower__7') + Q('VehPower__8') +
                          Q('VehPower__9') + Q('VehPower__10') + Q('VehPower__11') + Q('VehPower__12') + 
                          Q('VehAge__2') + Q('VehAge__3')
                           """

In [222]:
y_train, X_train = dmatrices(expr, claimsdf_train, return_type='dataframe')

- We define the train sets for the Logistic part of the model

In [223]:
y_train2, X_train2 = dmatrices(expr, claimsdf_train, return_type='dataframe')

In [224]:
X_train2.drop(columns=["Q('Area')", "Q('log_density')", "Q('VehPower__5')", "Q('VehPower__6')", "Q('VehPower__7')", "Q('VehPower__8')", "Q('VehPower__9')", "Q('VehPower__10')", "Q('VehPower__11')", "Q('VehPower__12')"], inplace=True)

In [228]:
zip_model2 = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train2, exposure=claimsdf_train.Exposure, inflation='logit').fit()

/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.211026
         Iterations: 35
         Function evaluations: 45
         Gradient evaluations: 45


/home/julian/miniconda3/envs/ironcon/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [226]:
print(zip_model2.summary())

                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:            Q('ClaimNb')   No. Observations:               542280
Model:             ZeroInflatedPoisson   Df Residuals:                   542265
Method:                            MLE   Df Model:                           14
Date:                 Tue, 05 Oct 2021   Pseudo R-squ.:                 0.02340
Time:                         12:54:51   Log-Likelihood:            -1.1444e+05
converged:                       False   LL-Null:                   -1.1718e+05
Covariance Type:             nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
inflate_Intercept               0.4079      0.056      7.249      0.000       0.298       0.518
inflate_Q('BonusMalus')        -0.0141      0.001    -17.704      0.000 

In [247]:
zip_model2.aic

228900.74683430497

#### ROOT MEAN SQUARE ERROR OF PREDICTION:

In [230]:
y_test, X_test = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [244]:
y_test2, X_test2 = dmatrices(expr, claimsdf_test, return_type='dataframe')

In [245]:
X_test2.drop(columns=["Q('Area')", "Q('log_density')", "Q('VehPower__5')", "Q('VehPower__6')", "Q('VehPower__7')", "Q('VehPower__8')", "Q('VehPower__9')", "Q('VehPower__10')", "Q('VehPower__11')", "Q('VehPower__12')"], inplace=True)

In [246]:
zip_model2_pred = zip_model2.predict(X_test, exog_infl=X_test2, exposure=claimsdf_test.Exposure)

- Let's create a dataframe with the predctions of the Zip GLM:

In [249]:
zip2_pred_df = pd.DataFrame(y_test)
zip2_pred_df['predicted_ClaimNb'] = zip_model2_pred

In [250]:
rmse_zip2 = []
for _ in range(10):
    zip2 = zip2_pred_df.sample(n=135733)
    rmse_ = ((((zip2["Q('ClaimNb')"] - zip2['predicted_ClaimNb']) ** 2).sum())/13574) ** 0.5
    rmse_zip2.append(rmse_)    

In [252]:
zip2_rmse = sum(rmse_zip2)/len(rmse_zip2)
zip2_rmse

0.744998169692998

### MODEL SELECTION

- The likelihood ratio test based on Posisson deviance can be applied recursively to a sequence of nested models. This leads to a step-wise reduction of model complexity, this is similar in spirit to the analysis of variance (ANOVA) in Listing 2.7, and it is often referred to as backward model selection